In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab12.ipynb")

# Lab 12: Climate data, CO2 and xarray

## Due Date

This assignment is due on **Wednesday November 17th, 2021, at 11:59PM PDT**.

## Collaboration Policy

Data science is a collaborative activity. While you may talk with others about
the homework, we ask that you **write your solutions individually**. If you do
discuss the assignments with others please **include their names** at the top
of your solution.

This lab accompanies a lecture for Berkeley's Data 100 that covers the fundamental physical mechanisms behind global warming and analyzes CO2 and ocean temperature data.

Authors: Fernando Pérez and [Dr. Chelle Gentemann](https://cgentemann.github.io).

**Collaborators:** *list names here*

In [1]:
# Run this cell to set up your notebook
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

from pathlib import Path

# Small style adjustments for more readable plots
plt.style.use("seaborn-whitegrid")
plt.rcParams["figure.figsize"] = (8, 6)
plt.rcParams["font.size"] = 14

# more readable exceptions
%pip install --quiet iwut
%load_ext iwut
%wut on

## Loading the Mauna Loa CO2 data

We start by loading the same dataset we used during the lecture, containing CO2 measurements in Mauna Loa, Hawaii.

In [2]:
# Set up to load data either from the shared directory that has the complete set,
# or from the local small copy available in the repo

DATA_DIR = Path('./data')

# If you want to run this on the Berkeley data hub, where we have a larger version of the data,
# uncomment the below. But do not do that for submitting the Lab, as the version that will
# run on the grader needs to use the path above and only has enough data to grade the Lab.
#DATA_DIR = Path.home()/Path('shared/climate-data')

Now we start by loading the CO2 data from a CSV file:

In [3]:
co2_file = DATA_DIR / "monthly_in_situ_co2_mlo_cleaned.csv"
data = pd.read_csv(co2_file)
data.head()

In [4]:
# get rid of missing values that are set to -99.99 and then drop from the dataset
data = pd.read_csv(co2_file, na_values=-99.99).dropna()
data.head()

In [5]:
# We did this in lecture, as a reminder this is what the data looks like:
plt.plot("fraction_date", "c02", data=data)
plt.plot("fraction_date", "data_adjusted_seasonally_fit", data=data);

# Exploring the annual anomaly

We are going to try to understand the annual variability on top of the growing trend, and see whether that variability is itself changing over time or not.

The figure above shows an annual cycle, alongside with perhaps some variability in it. As we saw in lecture (recall the super computer movie in talk with cities release of co2 strongest in winter), plants take up CO2 in (northern) spring/summer then release in fall/winter --- so the release is getting stronger. 

Let's try to estimate the increase in amplitude of annual cycle.

<!-- why would using a rolling mean make it look like the cycle is getting larger - both summer/winter rather than just winter? with this data can you figure out if summer or winter causing increase? -->

In the lecture notebook, we created the following figure, based on a quick and simple groupby operation and removal of the annual mean:

In [6]:
# calculate the annual cycle using groupby
annual = data.groupby(data.month).mean()
# calculate the anomaly
anomaly = annual - annual.mean()

# plot
fig, ax = plt.subplots()

ax.plot("fraction_date", "data_filled", "r.", data=data)
ax.plot("fraction_date", "data_adjusted_seasonally_fit", data=data)
ax.set_xlabel("Year")
ax.set_ylabel("CO2 fraction in dry air (ppm)")
ax.set_title("Monthly Mean CO2")
ax.grid(False)

axin1 = ax.inset_axes([0.1, 0.5, 0.35, 0.35])
axin1.plot(anomaly.c02, "b")
axin1.plot(anomaly.c02, "r.")
axin1.set_title("Seasonal Anomaly");

But if you look closely, that figure isn't quite the same as the one shown in slide 9 of the lecture:

<img src="annual-anomaly-orig.png" width="700px" />

We'll start by trying to understand the monthly data. The following shows us the data for all the years, by month:

In [7]:
plt.scatter('month', 'c02', data=data);

Unfortunately it's hard to see what's actually going on here.

### Question 1

Recreate the following figure, that shows the monthly cycle for all the years in the dataset:

<img src="monthly-cycles-co2.png" width="700px" />

Hint: Use `sns.lineplot(...)`

<!--
BEGIN QUESTION
name: q1
-->

In [8]:
...

### Question 2

Next, in order to recreate the figure in the talk, we're going to find the monthly anomaly averaging across years. You should end up with the following data frame after writing your function and running the cell (only the first few months are shown, it should have data for all 12 months):

<img src="monthly-co2-anomaly-df.png" width="150px" />

<!--
BEGIN QUESTION
name: q2
-->

In [9]:
def detrend(f):
    detrended_xarr = ...
    ...
    return detrended_xarr

dy = data.groupby('year')
c02anomaly = dy.apply(detrend)
display(c02anomaly)
monthly_anomaly = c02anomaly.groupby('month').mean()[['c02']]
monthly_anomaly

In [ ]:
grader.check("q2")

### Question 3

Next, recreate the following figure, which is much closer to the one in the lecture:

<img src="annual-anomaly-new.png" width="700px" />

**Hints:** 
- in order to get that smooth curve, you'll need to use `from scipy.interpolate import CubicSpline`. You can find the documentation for `CubicSpline` [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.CubicSpline.html).
- The "Seasonal Anomaly" inset graph can be created with `ax.inset_axes(...)`. You can treat the `Axes` object returned like any other `Axes` object.
- Remember that you can set the tick labels with `ax.set_xticks(...)`.
- To get the data points plotted as red dots, when you use `ax.plot(...)`, pass in `"r."` as the argument immediately following any data you pass in. The `"r"` indicates the color, while the `"."` indicates how the data will be plotted (if you don't put the `"."`, it will be a line instead).

<!--
BEGIN QUESTION
name: q3
-->

In [11]:
from scipy.interpolate import CubicSpline

fig, ax = plt.subplots()

...

# Exploring Snow with ERA 5

Now, we turn our attention to the global ERA 5 dataset and explore some other questions - it's a very rich and interesting dataset, and the lecture only scratched its surfac!

We are going today to focus on just one more bit: we'll take a look at the snow accumulation data for the northern and southern hemispheres.

But you should see this as an invitation to keep learning from these data! Think of looking at other variables in the dataset. Is there annual cycle? trend? Some of the data might look very different than the air temperature - eg. precipitation which is either 0 or +. Can you use PDFs to look at changes in distributions over a region? at a point? Or talk about the data a little & what you understand it is measuring? Are any of the data variables related to each other? Can you plot correlations between data?

In [12]:
import xarray as xr

monthly_2deg_path = DATA_DIR / "era5_snow_monthly_2deg_aws_v20210920.nc"

# The Berkeely hub has a version with not only snow data but many more variables.
# You can explore the full dataset by uncommenting the following  if you are on the
# campus hub. Do NOT do that for your lab submission, as that data is NOT available
# on Gradescope.
# monthly_2deg_path = DATA_DIR / "era5_monthly_2deg_aws_v20210920.nc"

ds = xr.open_dataset(monthly_2deg_path)
ds

In [13]:
mean_map = ds.mean("time")  # takes the mean across all variables in ds
mean_map.snow_density.plot();

In [14]:
snow = ds.snow_density
snow

### Question 4

Let's look at the snow across the globe in February and August, which are roughly the peaks of the summer/winter seasons.

Recreate the following figure, along with a corresponding one for August 1980:

<img src="snow-distribution-1980-02.png" width="500px" />

<!--
BEGIN QUESTION
name: q4
-->

In [15]:
...

In [16]:
...

### Question 5

Let's now find the peaks in the northern and southern hemispheres.

<!--
BEGIN QUESTION
name: q5
-->

In [17]:
t_snow_north = ...
t_snow_north.plot();

In [18]:
t_snow_south = ...
t_snow_south.plot();

### Question 6

Let's look for example at what the cycle in the northern hemisphere looks like for the year 1979. You need to replicate this figure:

<img src="snow-cycle-1979.png" width="500px" />

<!--
BEGIN QUESTION
name: q6
-->

In [19]:
...

### Question 7

And finally, we're going to find the peaks for the maximum and minimum snow accumulation in both hemispheres, and study when those peaks happen, how much snow there is, and whether the amounto of total snow is going up or down.

You'll need to recreate a dataframe like the one pictured here (only a few rows shown, and this is for the northern hemisphere, you'll make one for each hemisphere):

<img src="snow-peaks-north-df.png" width="900px" />

**Hint:** look up the documentation for the xarray `idxmax` method.

**Hint:**
1. First, we want to extract the years from the data and iterate over them (try using `range`).
1. When iterating over the years, we want to first pick the designated year, then find the days with the most and least snow as well as the amount of snow on those days.
1. Once we've found the days, assign `vals` to be an array of the information we've calculated.
1. Finally, append all the necessary information to `peaks`.

- Amplitude is `max - min`
- You may want to use `x.values.item` to extract information from the datetime objects.

<!--
BEGIN QUESTION
name: q7
-->

In [23]:
def extract_peaks(snow_data):
    ...
    years = ...
    peaks = []
    for y in years:
    ...
        vals = ...
    ...

    snow_peaks = pd.DataFrame(peaks, 
                              columns = ['year', 'min_date', 'max_date',
                                         'min_dayofyear', 'max_dayofyear',
                                         'min_snow', 'max_snow', 'amplitude'])

    return snow_peaks

peaks_north = extract_peaks(t_snow_north)
peaks_north.head()

With this data, we can now quickly explore several questions. For example, during what day of the year do the min and max happen?

In [22]:
plt.plot('year', 'min_dayofyear', data=peaks_north);
plt.plot('year', 'max_dayofyear', data=peaks_north);
plt.legend();

What is the min and max amount of snow at those times?

In [23]:
plt.plot('year', 'min_snow', data=peaks_north);
plt.plot('year', 'max_snow', data=peaks_north);
plt.legend();

Given the difference in scales, it's a bit easier to see what is happening if we plot the min and max separately:

In [24]:
plt.plot('year', 'min_snow', data=peaks_north);
plt.plot('year', 'min_snow', 'r.', data=peaks_north);
plt.legend();

In [25]:
plt.plot('year', 'max_snow', data=peaks_north);
plt.plot('year', 'max_snow', 'r.', data=peaks_north);
plt.legend();

What do you think this is telling us about the ice in the northern hemisphere? What are the main sources of ice there, and what is happening to them?

The above figures are for the northern hemisphere. Repeat them for the southern one. What do you observe? Any differences?

I hope this lab has taught you some useful skills regarding tools like xarray and data with a different structure than data frames. But more importantly, that it has shown you how the ideas from this course also apply to problems as critical to humanity as climate change, and that you have the foundations on which to better understand these questions.

Don't stop here! This dataset is fascinating, and both the lecture and the lab were barely an appetizer!

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)